<a href="https://colab.research.google.com/github/Yin-Tzu/predict_DA_reaction_product2/blob/main/3divided_straight_Cnumber/1differentCNN_3D20C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
file_dir = './2to16_Divideequally1200X200X200.zip'
zipFile = zipfile.ZipFile(file_dir)
for file in zipFile.namelist():
    zipFile.extract(file, '/content')  # 解压路径
zipFile.close()

In [ ]:
!pip install psutil

In [ ]:
import os
import psutil

# 顯示當前 python 程式佔用的記憶體大小
def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)
   
    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))

In [ ]:
import gc
def generator(C, r, batch_size):
    samples_per_epoch = C.shape[0]
    number_of_batches = samples_per_epoch / batch_size
    counter = 0
    while 1:
        X_batch = np.array(C[batch_size * counter:batch_size * (counter + 1)])
        y_batch = np.array(r[batch_size * counter:batch_size * (counter + 1)])
        #print(y_batch[0])
        y_batchOneHot = tf.keras.utils.to_categorical(y_batch,3)
        counter += 1
        # restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0
        yield X_batch, y_batchOneHot
        gc.collect()

In [ ]:
import zipfile
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import cv2
import copy
from sklearn.model_selection import train_test_split
xy=np.load('2to20_Divideequally1200X300X300.zip')
X_train, y_train_label, X_test, y_test_label,X_valid,y_valid_label=xy['X_train'],xy['y_train_label'],xy['X_test'],xy['y_test_label'],xy['X_valid'],xy['y_valid_label']
"""
z = zipfile.ZipFile("2to6_Divideequally1200X200X200.zip", "r")
#列印zip檔案中的檔案列表
for filename in z.namelist( ):
  print('File:', filename)
#讀取zip檔案中的第一個檔案
first_file_name = z.namelist()[0]
content = z.read(first_file_name)
print(first_file_name)
print(content)
"""

'\nz = zipfile.ZipFile("2to6_Divideequally1200X200X200.zip", "r")\n#列印zip檔案中的檔案列表\nfor filename in z.namelist( ):\n  print(\'File:\', filename)\n#讀取zip檔案中的第一個檔案\nfirst_file_name = z.namelist()[0]\ncontent = z.read(first_file_name)\nprint(first_file_name)\nprint(content)\n'

In [ ]:
y_TrainOneHot = tf.keras.utils.to_categorical(y_train_label)  # One-Hot编码
y_TestOneHot = tf.keras.utils.to_categorical(y_test_label)
y_ValidOneHot = tf.keras.utils.to_categorical(y_valid_label)  # One-Hot编码
print(X_train.shape, y_train_label.shape, X_test.shape, y_test_label.shape,X_valid.shape,y_valid_label.shape)
tStart = time.time()#計時開始

model = tf.keras.models.Sequential()  # 调用Sequential模型
model.add(layers.Conv2D(input_shape=(300,300, 3), filters=16, kernel_size=(3,3), kernel_initializer='TruncatedNormal', strides=1, padding='same', activation='relu', name='conv1'))  # 10*10
model.add(layers.AveragePooling2D(pool_size=(3,3), strides=2, padding='same', name='pool1'))  # 5*5
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), kernel_initializer='TruncatedNormal', strides=1, padding='same', activation='relu', name='conv3'))  # 10*10
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same', name='pool3'))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), kernel_initializer='TruncatedNormal', strides=1, padding='same', activation='relu', name='conv2'))  # 10*10
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same', name='pool2'))  # 5*5
#model.add(layers.Flatten(name='flatten'))
#model.add(layers.Dense(units=64, kernel_initializer='TruncatedNormal', activation='relu'))
model.add(layers.GlobalAveragePooling2D())
#model.add(layers.Dropout(0.3))  #在 0 和 1 之间浮动。需要丢弃的输入比例。
model.add(layers.Dense(units=10,kernel_initializer='TruncatedNormal', activation='softmax'))#,input_dim=100

print(model.summary())
#batch_size = 2
# 模型的训练 编译模型
opt = tf.keras.optimizers.Adam(learning_rate=0.0003)# 3*3
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])  # metrics是训练和测试期间的模型评估标准。

# 监控val_loss，当连续40轮变化小于0.0001时启动early stopping
#es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40, min_delta=0.0001)

# 训练模型
train_history = model.fit(x=X_train, y=y_TrainOneHot, validation_data=(X_valid, y_ValidOneHot), epochs=200, batch_size=2, verbose=2)
#train_history = model.fit_generator(generator(X_train, y_train_label, batch_size), epochs=100, steps_per_epoch=len(X_train)// batch_size,validation_data=generator(X_valid, y_valid_label, batch_size),validation_steps=len(X_valid) // batch_size, verbose=2,workers=5, use_multiprocessing=True)#validation_data=generator(X_test, y_test_label, batch_size),validation_steps=len(X_test) // batch_size
# 查看训练过程，之前的训练步骤的值都保存在这里面。这里共有loss,accuracy,val_loss,val_accuracy四个参数
print(train_history.history)

# 將模型儲存至 HDF5 檔案中
model.save('my_model2.h5')  # creates a HDF5 file 'my_model.h5'

scores = model.evaluate(X_test,y_TestOneHot)
#scores = model.evaluate_generator(generator(X_test, y_test_label, batch_size),steps=len(X_test)// batch_size)
print('loss, accuracy=',scores) #显示测试准确率[1]

prediction = model.predict_classes(X_test)
# 返回预测属于某标签的概率
y_score = model.predict_proba(X_test)

t2 = time.time()#計時結束
#列印結果
print("It cost %f sec" % (t2 - tStart))  #會自動做近位
print(t2 - tStart)  #原型長這樣

(7200, 300, 300, 3) (7200,) (2400, 300, 300, 3) (2400,) (2400, 300, 300, 3) (2400,)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 300, 300, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 300, 16)      64        
_________________________________________________________________
pool1 (AveragePooling2D)     (None, 150, 150, 16)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 150, 150, 32)      4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
pool3 (MaxPooling2D)         (None, 75, 75, 32)        0         
____________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


It cost 3554.452813 sec
3554.4528131484985


準確率loss與epoch作圖及混淆矩陣

In [ ]:

prediction = model.predict_classes(X_test)
def show_train_history(train_history, train, validation):  # 訓練集驗證準確度對epoch做圖
    plt.plot(train_history.history[train])  # 绘制训练数据的执行结果
    plt.plot(train_history.history[validation])  # 绘制验证数据的执行结果
    plt.title('Train History')  # 图标题
    plt.xlabel('epoch')  # x轴标签
    plt.ylabel(train)  # y轴标签
    plt.legend(['train', 'validation'], loc='upper left')  # 添加左上角图例
    plt.show()

show_train_history(train_history, 'accuracy', 'val_accuracy')

show_train_history(train_history, 'loss', 'val_loss')

print(pd.crosstab(y_test_label,prediction,rownames=['label'],colnames=['predict']))   # https://zhuanlan.zhihu.com/p/52368125  其他呈現交叉表方法(平均、彩色圖...)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
import cv2


im = cv2.imread('20-1.png')
im = cv2.resize(im, (300,300), interpolation=cv2.INTER_AREA)
im=im[np.newaxis, :]
prediction = model.predict_classes(im)
print(prediction)

[8]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
